In [8]:
import numpy as np
import pandas as pd
import os
import random
import warnings
from sklearn.base import clone
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import VotingRegressor
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from scipy.optimize import minimize
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
from colorama import Fore, Style
from IPython.display import clear_output
from lightgbm import LGBMRegressor, LGBMClassifier
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Create analysis_output folder if it doesn't exist
output_folder = 'analysis_output'
os.makedirs(output_folder, exist_ok=True)

# Load data functions
def process_file(filename, dirname):
    df = pd.read_parquet(os.path.join(dirname, filename, 'part-0.parquet'))
    df.drop('step', axis=1, inplace=True)
    return df.describe().values.reshape(-1), filename.split('=')[1]

def load_time_series(dirname) -> pd.DataFrame:
    ids = os.listdir(dirname)
    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(lambda fname: process_file(fname, dirname), ids), total=len(ids)))
    stats, indexes = zip(*results)
    df = pd.DataFrame(stats, columns=[f"stat_{i}" for i in range(len(stats[0]))])
    df['id'] = indexes
    return df

# Set display all columns in dataframes property
pd.options.display.max_columns = None

In [3]:
# Load data
train_data = pd.read_csv('input/train.csv')
test_data = pd.read_csv('input/test.csv')
sample_data = pd.read_csv('input/sample_submission.csv')

# Load time series data
train_ts_data = load_time_series("input/series_train.parquet")
test_ts_data = load_time_series("input/series_test.parquet")

100%|██████████| 2/2 [00:00<00:00, 11.11it/s]


In [4]:
# Remove id column from time series data
time_series_columns = train_ts_data.columns.tolist()
time_series_columns.remove("id")

# Merge data
train_data = pd.merge(train_data, train_ts_data, how="left", on='id')
test_data = pd.merge(test_data, test_ts_data, how="left", on='id')
train_data = train_data.drop('id', axis=1)
test_data = test_data.drop('id', axis=1)

train_data.head()

,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Season,Fitness_Endurance-Max_Stage,Fitness_Endurance-Time_Mins,Fitness_Endurance-Time_Sec,FGC-Season,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-Season,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,BIA-BIA_BMR,BIA-BIA_DEE,BIA-BIA_ECW,BIA-BIA_FFM,BIA-BIA_FFMI,BIA-BIA_FMI,BIA-BIA_Fat,BIA-BIA_Frame_num,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_A-Season,PAQ_A-PAQ_A_Total,PAQ_C-Season,PAQ_C-PAQ_C_Total,PCIAT-Season,PCIAT-PCIAT_01,PCIAT-PCIAT_02,PCIAT-PCIAT_03,PCIAT-PCIAT_04,PCIAT-PCIAT_05,PCIAT-PCIAT_06,PCIAT-PCIAT_07,PCIAT-PCIAT_08,PCIAT-PCIAT_09,PCIAT-PCIAT_10,PCIAT-PCIAT_11,PCIAT-PCIAT_12,PCIAT-PCIAT_13,PCIAT-PCIAT_14,PCIAT-PCIAT_15,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii,stat_0,stat_1,stat_2,stat_3,stat_4,stat_5,stat_6,stat_7,stat_8,stat_9,stat_10,stat_11,stat_12,stat_13,stat_14,stat_15,stat_16,stat_17,stat_18,stat_19,stat_20,stat_21,stat_22,stat_23,stat_24,stat_25,stat_26,stat_27,stat_28,stat_29,stat_30,stat_31,stat_32,stat_33,stat_34,stat_35,stat_36,stat_37,stat_38,stat_39,stat_40,stat_41,stat_42,stat_43,stat_44,stat_45,stat_46,stat_47,stat_48,stat_49,stat_50,stat_51,stat_52,stat_53,stat_54,stat_55,stat_56,stat_57,stat_58,stat_59,stat_60,stat_61,stat_62,stat_63,stat_64,stat_65,stat_66,stat_67,stat_68,stat_69,stat_70,stat_71,stat_72,stat_73,stat_74,stat_75,stat_76,stat_77,stat_78,stat_79,stat_80,stat_81,stat_82,stat_83,stat_84,stat_85,stat_86,stat_87,stat_88,stat_89,stat_90,stat_91,stat_92,stat_93,stat_94,stat_95
0,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fall,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,7.0,0.0,6.0,0.0,6.0,1.0,Fall,2.0,2.66855,16.8792,932.498,1492.00,8.25598,41.5862,13.8177,3.06143,9.21377,1.0,24.4349,8.89536,38.9177,19.5413,32.6909,NaN,NaN,NaN,NaN,Fall,5.0,4.0,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,4.0,0.0,4.0,4.0,4.0,4.0,4.0,4.0,2.0,4.0,55.0,NaN,NaN,NaN,Fall,3.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Summer,9,0,NaN,NaN,Fall,14.035590,48.0,46.0,22.0,75.0,70.0,122.0,NaN,NaN,NaN,NaN,Fall,3.0,0.0,NaN,NaN,NaN,NaN,5.0,0.0,11.0,1.0,11.0,1.0,3.0,0.0,Winter,2.0,2.57949,14.0371,936.656,1498.65,6.01993,42.0291,12.8254,1.21172,3.97085,1.0,21.0352,14.97400,39.4497,15.4107,27.0552,NaN,NaN,Fall,2.340,Fall,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Fall,46.0,64.0,Summer,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,NaN,65.0,94.0,117.0,Fall,5.0,7.0,33.0,Fall,20.0,1.0,10.2,1.0,14.7,2.0,7.0,1.0,10.0,1.0,10.0,1.0,5.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Summer,2.170,Fall,5.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,2.0,2.0,1.0,1.0,28.0,Fall,38.0,54.0,Summer,2.0,0.0,NaN,NaN,NaN,NaN,NaN,

In [5]:
# Isolate the physical attribute columns for analysis
physical_columns = [
    'Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex',
    'CGAS-Season', 'CGAS-CGAS_Score', 'Physical-Season', 'Physical-BMI',
    'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
    'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP'
]

In [6]:
# Function to analyze columns
def analyze_column(column):
    total_count = len(train_data)
    missing_count = train_data[column].isnull().sum()
    missing_percentage = (missing_count / total_count) * 100
    unique_values = train_data[column].nunique()
    
    if pd.api.types.is_numeric_dtype(train_data[column]):
        mean_value = train_data[column].mean()
        median_value = train_data[column].median()
        std_dev = train_data[column].std()
        return {
            "Column": column,
            "Total Count": total_count,
            "Missing Count": missing_count,
            "Missing Percentage": f"{missing_percentage:.2f}%",
            "Unique Values": unique_values,
            "Data Type": train_data[column].dtype,
            "Mean": mean_value,
            "Median": median_value,
            "Standard Deviation": std_dev
        }
    else:
        top_values = train_data[column].value_counts().head(3).to_dict()
        return {
            "Column": column,
            "Total Count": total_count,
            "Missing Count": missing_count,
            "Missing Percentage": f"{missing_percentage:.2f}%",
            "Unique Values": unique_values,
            "Data Type": train_data[column].dtype,
            "Top 3 Values": top_values
        }
        
column_profiles = [analyze_column(col) for col in physical_columns]
column_profiles_df = pd.DataFrame(column_profiles)

# Save column profiles to CSV
column_profiles_df.to_csv(os.path.join(output_folder, 'column_profiles.csv'), index=False)
print(f"Column profiles saved to {os.path.join(output_folder, 'column_profiles.csv')}")

Column profiles saved to analysis_output\column_profiles.csv


In [9]:
# Visualize missing data
plt.figure(figsize=(12, 6))
sns.heatmap(train_data[physical_columns].isnull(), cbar=False, yticklabels=False, cmap='viridis')
plt.title('Missing Data in Physical Attribute Columns')
plt.xlabel('Columns')
plt.ylabel('Rows')
plt.tight_layout()
plt.savefig(os.path.join(output_folder, 'missing_data_heatmap.png'))
plt.close()
print(f"Missing data heatmap saved to {os.path.join(output_folder, 'missing_data_heatmap.png')}")

Missing data heatmap saved to analysis_output\missing_data_heatmap.png


In [10]:
# Correlation matrix for numeric columns
numeric_columns = train_data[physical_columns].select_dtypes(include=[np.number]).columns
correlation_matrix = train_data[numeric_columns].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Matrix of Numeric Physical Attributes')
plt.tight_layout()
plt.savefig(os.path.join(output_folder, 'correlation_matrix.png'))
plt.close()
print(f"Correlation matrix saved to {os.path.join(output_folder, 'correlation_matrix.png')}")

Correlation matrix saved to analysis_output\correlation_matrix.png


In [11]:
# Additional analysis: Distribution plots for numeric columns
for column in numeric_columns:
    plt.figure(figsize=(10, 6))
    sns.histplot(train_data[column].dropna(), kde=True)
    plt.title(f'Distribution of {column}')
    plt.xlabel(column)
    plt.ylabel('Count')
    plt.tight_layout()
    plt.savefig(os.path.join(output_folder, f'{column}_distribution.png'))
    plt.close()
    print(f"Distribution plot for {column} saved to {os.path.join(output_folder, f'{column}_distribution.png')}")

print("All analyses completed and saved to the 'analysis_output' folder.")

Distribution plot for Basic_Demos-Age saved to analysis_output\Basic_Demos-Age_distribution.png
Distribution plot for Basic_Demos-Sex saved to analysis_output\Basic_Demos-Sex_distribution.png
Distribution plot for CGAS-CGAS_Score saved to analysis_output\CGAS-CGAS_Score_distribution.png
Distribution plot for Physical-BMI saved to analysis_output\Physical-BMI_distribution.png
Distribution plot for Physical-Height saved to analysis_output\Physical-Height_distribution.png
Distribution plot for Physical-Weight saved to analysis_output\Physical-Weight_distribution.png
Distribution plot for Physical-Waist_Circumference saved to analysis_output\Physical-Waist_Circumference_distribution.png
Distribution plot for Physical-Diastolic_BP saved to analysis_output\Physical-Diastolic_BP_distribution.png
Distribution plot for Physical-HeartRate saved to analysis_output\Physical-HeartRate_distribution.png
Distribution plot for Physical-Systolic_BP saved to analysis_output\Physical-Systolic_BP_distribut

In [14]:
# Supplement missing data with data from WHO
# Load who data and group
def load_who_bmi_data(file_path):
    who_data = pd.read_csv(file_path)
    # Group by age and sex, taking the mean of other columns
    who_data = who_data.groupby(['age', 'sex']).agg({
        'L': 'mean', 'mean_bmi': 'mean', 'S': 'mean',
        'SD4neg': 'mean', 'SD3neg': 'mean', 'SD2neg': 'mean', 'SD1neg': 'mean',
        'SD0': 'mean', 'SD1': 'mean', 'SD2': 'mean', 'SD3': 'mean', 'SD4': 'mean'
    }).reset_index()
    who_data = who_data.set_index(['sex', 'age'])
    return who_data

who_bmi_data = load_who_bmi_data('supplemental_data/bmi_for_age_5_to_19.csv')

who_bmi_data.head()

,,L,mean_bmi,S,SD4neg,SD3neg,SD2neg,SD1neg,SD0,SD1,SD2,SD3,SD4
sex,age,,,,,,,,,,,,
0,5,-0.854909,15.270855,0.085183,11.213727,12.119727,13.025545,14.065455,15.270727,16.683182,18.359182,20.377818,22.396455
1,5,-0.977127,15.249500,0.099205,10.767364,11.743818,12.720545,13.871909,15.249455,16.926909,19.014091,21.682273,24.350182
0,6,-1.111075,15.379233,0.088524,11.288917,12.186500,13.084167,14.134083,15.379167,16.881583,18.732417,21.073167,23.414083
1,6,-1.163800,15.320042,0.104477,10.749000,11.727917,12.707000,13.881583,15.320083,17.125583,19.466500,22.634333,25.802083
0,7,-1.348825,15.594317,0.092724,11.412000,12.313833,13.215667,14.289583,15.594333,17.218417,19.306583,22.109250,24.912167


In [16]:
# Impute WHO data
def get_who_bmi_stats(age, sex):
    try:
        stats = who_bmi_data.loc[(sex, age), ['mean_bmi', 'S']]
        return stats['mean_bmi'], stats['S']
    except KeyError:
        return None, None

def impute_bmi(age, sex):
    mean_bmi, sd = get_who_bmi_stats(age, sex)
    if mean_bmi is not None and sd is not None:
        # Generate a random BMI value from a normal distribution
        imputed_bmi = np.random.normal(mean_bmi, sd)
        return round(imputed_bmi, 2)
    else:
        return None
    
def apply_bmi_imputation(df):
    def impute_if_missing(row):
        if pd.isna(row['Physical-BMI']):
            age = row['Basic_Demos-Age']
            sex = row['Basic_Demos-Sex']  # 0 for male, 1 for female
            if 5 <= age <= 19:
                return impute_bmi(age, sex)
            else:
                return None  # or you could implement a different imputation method for other ages
        else:
            return row['Physical-BMI']
    
    df['Physical-BMI'] = df.apply(impute_if_missing, axis=1)
    return df

In [17]:
# Apply and check results
train_data = apply_bmi_imputation(train_data)
test_data = apply_bmi_imputation(test_data)

print("Number of missing BMI values after imputation:")
print(train_data['Physical-BMI'].isna().sum())
print("\nSample of imputed BMI values:")
print(train_data[train_data['Physical-BMI'].notnull()].sample(5)[['Basic_Demos-Age', 'Basic_Demos-Sex', 'Physical-BMI']])

Number of missing BMI values after imputation:
21

Sample of imputed BMI values:
      Basic_Demos-Age  Basic_Demos-Sex  Physical-BMI
2221                9                1     14.767160
19                 10                0     16.820000
3446                8                0     17.353403
210                 5                0     15.968632
1841                8                1     19.180263


In [18]:
# Additional analysis of imputation results
print("\nImputation summary by age:")
age_summary = train_data.groupby('Basic_Demos-Age').agg({
    'Physical-BMI': ['count', 'mean', 'std', 'min', 'max'],
    'Basic_Demos-Sex': 'count'
})
print(age_summary)


Imputation summary by age:
                Physical-BMI                                             \
                       count       mean       std        min        max   
Basic_Demos-Age                                                           
5                        112  16.643384  2.714359  12.853139  26.512004   
6                        369  16.235721  2.603852   0.000000  34.055363   
7                        436  16.490898  2.852864   9.693766  43.468833   
8                        490  17.301484  3.607021   0.000000  59.132048   
9                        467  17.509783  3.137871  11.925153  33.729946   
10                       420  18.393221  4.042838   0.000000  44.554097   
11                       334  19.048164  3.976348  11.915254  37.484035   
12                       291  20.113377  4.432500  14.223937  40.399621   
13                       236  20.464829  4.768933   0.000000  39.494148   
14                       200  21.756331  5.142430  13.740613  44.835548 